In [2]:
from t_smiles.DataSet.STDTokens import CTokens, STDTokens_Frag_File
from t_smiles.MolUtils.RDKUtils.Frag.RDKFragUtil import Fragment_Alg

from t_smiles.DataSet.Graph.CNJMolAssembler import CNJMolAssembler

from t_smiles.DataSet.Graph.CNJMolUtil import CNJMolUtil
from t_smiles.DataSet.Graph.CNJTMol import CNJMolUtils

Test encoding algorithm

In [5]:
# celecoxib
smls = "CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F"  # type: str
# LSD
smls = "CCN(CC)C(=O)[C@H]1CN([C@@H]2CC3=CNC4=CC=CC(=C34)C2=C1)C"  # type: str


dec_algs = [
    Fragment_Alg.Vanilla,
    Fragment_Alg.JTVAE,
    Fragment_Alg.BRICS,
    Fragment_Alg.BRICS_Base,
    Fragment_Alg.MMPA,
    Fragment_Alg.Scaffold,
    Fragment_Alg.BRICS_DY,
    Fragment_Alg.MMPA_DY,
    Fragment_Alg.Scaffold_DY,
    # Fragment_Alg.RBrics_DY,
]

print("[smls is]:", smls)

ctoken = CTokens(STDTokens_Frag_File(None), max_length=256, invalid=True, onehot=False)


for dec_alg in dec_algs:
    combine_sml, combine_smt, amt_bfs_smarts = CNJMolUtils.encode_single(
        smls, ctoken, dec_alg
    )  # TODO: This returns a tuple of length 3, are the variables correctly assigned?

    print("[dec_alg is]:", dec_alg.name)
    print("[TSSA/TSDY]:", combine_sml)
    print("[TSID     ]:", combine_smt)

[smls is]: CCN(CC)C(=O)[C@H]1CN([C@@H]2CC3=CNC4=CC=CC(=C34)C2=C1)C
[dec_alg is]: Vanilla
[TSSA/TSDY]: CCN(CC)C(=O)[C@@H]1C=C2C3=C4C(=CNC4=CC=C3)C[C@H]2N(C)C1&&&
[TSID     ]: CCN(CC)C(=O)[C@@H]1C=C2C3=C4C(=CNC4=CC=C3)C[C@H]2N(C)C1&&&
[ChemUtils.get_clique_mol].exception[kekuleSmiles=True]: ("Can't kekulize mol.  Unkekulized atoms: 10 11 12",)
[MolTree.init].Exception: ("cannot access local variable 'new_mol' where it is not associated with a value",)
[MolTree.init].Exception-smiles: CCN(CC)C(=O)[C@H]1CN([C@@H]2CC3=CNC4=CC=CC(=C34)C2=C1)C
[CNJMolUtils.encode_single].exception: ("'CNJTMolTree' object has no attribute 'amt_bfs_smarts'",)
[dec_alg is]: JTVAE
[TSSA/TSDY]: CC&&&
[TSID     ]: CC&&&
[dec_alg is]: BRICS
[TSSA/TSDY]: CC&CN&N&CN&CC^CN&&C&CC&C1=C2C3=C4C(=CNC4=CC=C3)C[C@H]2NCC1^C=O^CN&&&&&
[TSID     ]: CC&CN&N&CN&CC^CN&&C&CC&C1=C2C3=C4C(=CNC4=CC=C3)C[C@H]2NCC1^C=O^CN&&&&&
[dec_alg is]: BRICS_Base
[TSSA/TSDY]: CC&CN&N&CN&CC^CN&&C&C=O&&CC^CN1CCC=C2C3=C4C(=CNC4=CC=C3)C[C@H]21&&&
[TSID 

Test decoding Algorithm

In [4]:
maxlen = 512
vocab_file = r"../RawData/Chembl/Test/Chembl_test.smi.[MMPA_DY][237]_token.voc.smi"

ctoken = CTokens(
    STDTokens_Frag_File(vocab_file),
    is_pad=True,
    pad_symbol=" ",
    startend=True,
    max_length=maxlen,
    flip=False,
    invalid=True,
    onehot=False,
)

# SMILES	 = 'CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F' #Celecoxib
# bfs_ex = 'CC1=CC=C(C=C1)C2=CC(=NN2C3=CC=C(C=C3)S(=O)(=O)N)C(F)(F)F&&&'	                        #TS_Vanilla

# bfs_ex = 'CC&C1=CC=CC=C1&CC&C1=C[NH]N=C1&CC&C^CN^CF&C1=CC=CC=C1&&CF^CS&&CF^S&&&NS&&O=S&O=S&&'	#TSSA-J
# bfs_ex = 'CC1=CC=C(C2=CC(C(F)(F)F)=NN2C2=CC=C(S(N)(=O)=O)C=C2)C=C1&&&'	                        #TSSA-B
# bfs_ex = 'CC&C1=CC=CC=C1&CC&C1=C[NH]N=C1&CN&C1=CC=CC=C1^CC^CS&C&N[SH](=O)=O&CF&&&&FCF&&'	    #TSSA-M
# bfs_ex = 'CC&C1=CC=C(C2=CC=NN2C2=CC=CC=C2)C=C1&CC&FC(F)F^CS&&N[SH](=O)=O&&&'	                #TSSA-S

# bfs_ex = 'CC1=CC=C(C2=CC(C(F)(F)F)=NN2C2=CC=C(S(N)(=O)=O)C=C2)C=C1&&&'	                        #TSDY-B
bfs_ex = "*C&*C1=CC=C(*)C=C1&*C1=CC(*)=NN1*&*C(*)(F)F&*F^*C1=CC=C(*)C=C1&&*S(N)(=O)=O&&&"  # TSDY-M
# bfs_ex = '*C&*C1=CC=C(C2=CC(*)=NN2C2=CC=C(*)C=C2)C=C1&*C(F)(F)F&&*S(N)(=O)=O&&'	            #TSDY-S

# bfs_ex = 'CC1=CC=C(C2=CC(C(F)(F)F)=NN2C2=CC=C(S(N)(=O)=O)C=C2)C=C1&&&'	                                                        #TSID-B
# bfs_ex = '[1*]C&[1*]C1=CC=C([2*])C=C1&[2*]C1=CC([3*])=NN1[5*]&[3*]C([4*])(F)F&[4*]F^[5*]C1=CC=C([6*])C=C1&&[6*]S(N)(=O)=O&&&'	#TSID_M
# bfs_ex = '[1*]C&[1*]C1=CC=C(C2=CC([2*])=NN2C2=CC=C([3*])C=C2)C=C1&[2*]C(F)(F)F&&[3*]S(N)(=O)=O&&'	                            #TSID-S

# asm_alg = 'CALG_TSSA'
asm_alg = "CALG_TSDY"
# asm_alg = 'CALG_TSID'

bfs_ex = "".join(bfs_ex.strip().split(" "))
print("input:=", bfs_ex)


bfs_ex_smiles = CNJMolUtil.split_ex_smiles(bfs_ex, delimiter="^")
print("bfs_ex_smiles", bfs_ex_smiles)

n_samples = 1
for i in range(n_samples):
    re_smils, bfs_ex_smiles_sub, new_vocs_sub, _ = CNJMolAssembler.decode_single(
        bfs_ex, ctoken, asm_alg, n_samples=1, p_mean=None
    )  # TODO: This returns tuple of length 4 not 3. Are the variables correctly assigned?
    print("dec_smile:=", re_smils)

input:= *C&*C1=CC=C(*)C=C1&*C1=CC(*)=NN1*&*C(*)(F)F&*F^*C1=CC=C(*)C=C1&&*S(N)(=O)=O&&&
bfs_ex_smiles ['*C', '&', '*C1=CC=C(*)C=C1', '&', '*C1=CC(*)=NN1*', '&', '*C(*)(F)F', '&', '*F', '*C1=CC=C(*)C=C1', '&', '&', '*S(N)(=O)=O', '&', '&', '&']
[len of candidates]:6, select the first 3 as target
dec_smile:= Cc1ccc(-c2cc(-c3ccc(S(N)(=O)=O)cc3)nn2C(F)(F)F)cc1
